In [1]:
import numpy as np
import tensorflow as tf
import requests

C:\Users\T480\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# from https://github.com/m-colombo/Tensorflow-EchoStateNetwork/blob/master/esn_cell.py

from tensorflow.python.ops import rnn_cell_impl
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.framework.ops import convert_to_tensor


class ESNCell(rnn_cell_impl.RNNCell):
    """Echo State Network Cell.

    Based on http://www.faculty.jacobs-university.de/hjaeger/pubs/EchoStatesTechRep.pdf
    Only the reservoir, the randomized recurrent layer, is modelled. The readout trainable layer
    which map reservoir output to the target output is not implemented by this cell,
    thus neither are feedback from readout to the reservoir (a quite common technique).

    Here a practical guide to use Echo State Networks:
    http://minds.jacobs-university.de/sites/default/files/uploads/papers/PracticalESN.pdf

    Since at the moment TF doesn't provide a way to compute spectral radius
    of a matrix the echo state property necessary condition `max(eig(W)) < 1` is approximated
    scaling the norm 2 of the reservoir matrix which is an upper bound of the spectral radius.
    See https://en.wikipedia.org/wiki/Matrix_norm, the section on induced norms."""
    
    def __init__(self, num_units, wr2_scale=0.7, connectivity=0.1, leaky=1.0, activation=math_ops.tanh,
               win_init=init_ops.random_normal_initializer(),
               wr_init=init_ops.random_normal_initializer(),
               bias_init=init_ops.random_normal_initializer()):
        """Initialize the Echo State Network Cell.

        Args:
          num_units: Int or 0-D Int Tensor, the number of units in the reservoir
          wr2_scale: desired norm2 of reservoir weight matrix.
            `wr2_scale < 1` is a sufficient condition for echo state property.
          connectivity: connection probability between two reservoir units
          leaky: leaky parameter
          activation: activation function
          win_init: initializer for input weights
          wr_init: used to initialize reservoir weights before applying connectivity mask and scaling
          bias_init: initializer for biases
        """
        self._num_units = num_units
        self._leaky = leaky
        self._activation = activation
        
        def _wr_initializer(shape, dtype, partition_info=None):
            wr = wr_init(shape, dtype=dtype)

            connectivity_mask = math_ops.cast(
                  math_ops.less_equal(
                    random_ops.random_uniform(shape),
                    connectivity),
                dtype)

            wr = math_ops.multiply(wr, connectivity_mask)

            wr_norm2 = math_ops.sqrt(math_ops.reduce_sum(math_ops.square(wr)))

            is_norm_0 = math_ops.cast(math_ops.equal(wr_norm2, 0), dtype)

            wr = wr * wr2_scale / (wr_norm2 + 1 * is_norm_0)

            return wr
    
        self._win_initializer = win_init
        self._bias_initializer = bias_init
        self._wr_initializer = _wr_initializer
        
    

    

    @property
    def output_size(self):
        return self._num_units

    @property
    def state_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        
        """ Run one step of ESN Cell

            Args:
              inputs: `2-D Tensor` with shape `[batch_size x input_size]`.
              state: `2-D Tensor` with shape `[batch_size x self.state_size]`.
              scope: VariableScope for the created subgraph; defaults to class `ESNCell`.

            Returns:
              A tuple `(output, new_state)`, computed as
              `output = new_state = (1 - leaky) * state + leaky * activation(Win * input + Wr * state + B)`.

            Raises:
              ValueError: if `inputs` or `state` tensor size mismatch the previously provided dimension.
        """

        inputs = convert_to_tensor(inputs)
        input_size = inputs.get_shape().as_list()[1]
        dtype = inputs.dtype

        with vs.variable_scope(scope or type(self).__name__, reuse = F):  # "ESNCell"
            
            win = vs.get_variable("InputMatrix", [input_size, self._num_units], dtype=dtype,
                            trainable=False, initializer=self._win_initializer)
            wr = vs.get_variable("ReservoirMatrix", [self._num_units, self._num_units], dtype=dtype,
                           trainable=False, initializer=self._wr_initializer)
            b = vs.get_variable("Bias", [self._num_units], dtype=dtype, trainable=False, initializer=self._bias_initializer)

            in_mat = array_ops.concat([inputs, state], axis=1)
            weights_mat = array_ops.concat([win, wr], axis=0)

            output = (1 - self._leaky) * state + self._leaky * self._activation(math_ops.matmul(in_mat, weights_mat) + b)

        return output, output


In [3]:
def MackeyGlass(tr_size=500, washout_size=50, units=30, connectivity=0.2, scale=0.7, elements=2000):
    print("Fetching data...")
    data_str = requests.get('https://raw.githubusercontent.com/RomainPastureau/Reservoir-Jupyter/master/MackeyGlass_t17.txt').content
    
    #data = map(float, data_str.splitlines()[:elements])
    data = [float(s) for s in data_str.splitlines()[:elements]]
    data_t = tf.reshape(tf.constant(data), [1, elements, 1])
    esn = ESNCell(units, connectivity, scale)

    print("Building graph...")
    outputs, final_state = tf.nn.dynamic_rnn(esn, data_t, dtype=tf.float32)
    washed = tf.squeeze(tf.slice(outputs, [0, washout_size, 0], [-1, -1, -1]))

    with tf.Session() as S:
        S.run(tf.global_variables_initializer())

        print("Computing embeddings...")
        res = S.run(washed)

        print("Computing direct solution...")
        state = np.array(res)
        tr_state = np.mat(state[:tr_size])
        ts_state = np.mat(state[tr_size:])
        wout = np.transpose(np.mat(data[washout_size+1:tr_size+washout_size+1]) * np.transpose(np.linalg.pinv(tr_state)))

        print("Testing performance...")
        ts_out = np.mat((np.transpose(ts_state * wout).tolist())[0][:-1])
        ts_y = np.mat(data[washout_size+tr_size+1:])

        ts_mse = np.mean(np.square(ts_y - ts_out))

    print("Test MSE: " + str(ts_mse))

In [4]:
def get_data(filename,preproc = False):
    #get the data
    with open (filename) as f:
        txt_data = [x.split(';') for x in f.readlines()]
        tt = [x[3].strip() for x in txt_data[1:]]
        rf = [x[4].strip() for x in txt_data[1:]]
        data = np.array((tt,rf),dtype=float)
        if preproc:
            #scale the data to 0:1
            data[0,:] /= np.max(data[0,:])
            data[1,:] /= np.max(data[1,:])
            #whiten the data
            data[0,:] = (data[0,:] - np.mean(data[0,:]))/np.std(data[0,:])
            data[1,:] = (data[1,:] - np.mean(data[1,:]))/np.std(data[1,:])
    return data

In [5]:
def break_sequence(data,lookback):
    #data assumed to be shape (variables,timesteps)
    n = data.shape[1]
    windows = [data[:,i - lookback : i] for i in np.arange(lookback,n)]
    sequence = np.stack(windows,axis = 1)
    return sequence

In [6]:
def weather_esn():
    #get the (pre-processed) data
    filename = 'temperature_toy.txt'
    preproc = True
    #get the data
    with open (filename) as f:
        txt_data = [x.split(';') for x in f.readlines()]
        tt = [x[3].strip() for x in txt_data[1:]]
        rf = [x[4].strip() for x in txt_data[1:]]
        data = np.array((tt,rf),dtype=float)
        if preproc:
            #scale the data to 0:1
            data[0,:] /= np.max(data[0,:])
            data[1,:] /= np.max(data[1,:])
            #whiten the data
            data[0,:] = (data[0,:] - np.mean(data[0,:]))/np.std(data[0,:])
            data[1,:] = (data[1,:] - np.mean(data[1,:]))/np.std(data[1,:])
    #given the data, break it up into smaller sequences
    seq = break_sequence(data,25)
    #make a train and a test set
    train = seq[:,:100,:]
    test = seq[:,100:,:]
    #take the first elements of the sequence to predict the last
    x_train = train[:,:,:24]
    t_train = train[:,:,24]
    x_test = test[:,:,:24]
    t_test = test[:,:,24]
    #create the esn cell
    esn = ESNCell(50,0.2,0.7)
    #run
    with tf.Session() as S:
        S.run(tf.global_variables_initializer())
        #for each timestep
        state = tf.constant(0, shape = [100,50],dtype = tf.float64)
        for t in range(24):
            y, state = esn(x_train[:,:,t].T,state)
            variable.append(y)

In [10]:
variable = []
weather_esn()
for y in variable:
    print(y)

Tensor("ESNCell_24/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_25/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_26/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_27/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_28/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_29/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_30/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_31/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_32/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_33/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_34/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_35/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_36/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_37/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_38/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_39/add_1:0", shape=(100, 50), dtype=float64)
Tensor("ESNCell_40/add_1

In [ ]:
data = get_data('temperature_toy.txt',True)
seq = break_sequence(data,50)

In [ ]:
variable